In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import glob

file_pubs = r'E:\safety_sim\dataset\위치 데이터\서울시 유흥주점영업 인허가 정보 1904-2025.csv'

df_pubs = pd.read_csv(file_pubs, encoding="cp949")


df_pubs

df_cleaned = df_pubs[df_pubs["상세영업상태명"] != "폐업"]



In [ ]:
df_cleaned['구'] = df_cleaned['지번주소'].str.extract(r'서울특별시\s(.*?)\s')

df_counts = df_cleaned['구'].value_counts().reset_index()
df_counts.columns = ['구', '개수']

df_counts.head()

In [ ]:
import json
import folium
import pandas as pd
from branca.colormap import LinearColormap

# 색상 단계 정의
colors = ['red', 'lightcoral', 'orange', 'blue', 'lightblue']
color_scale = LinearColormap(colors=colors,
                             vmin=df_counts['개수'].min(),
                             vmax=df_counts['개수'].max())

# GeoJSON 파일 로드
geojson_path = r'E:\safety_sim\dataset\서울특별시.json'
with open(geojson_path, encoding='utf-8') as f:
    seoul_geo = json.load(f)

# GeoJSON에 'gu' 필드 추가
for feature in seoul_geo['features']:
    adm_nm = feature['properties']['adm_nm']  # 예: '서울특별시 성동구 행당동'
    gu_name = adm_nm.split()[1]  # '성동구'
    feature['properties']['gu'] = gu_name

# CCTV 데이터에서 자치구 필터링
geo_gu_list = {f['properties']['gu'] for f in seoul_geo['features']}
gu_counts_filtered = df_counts[df_counts['구'].isin(geo_gu_list)]

# folium 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

# 스타일 함수 정의 (색상 적용)
def style_function(feature):
    gu_name = feature['properties']['gu']
    value = gu_counts_filtered.loc[gu_counts_filtered['구'] == gu_name, '개수']
    if not value.empty:
        fill_color = color_scale(value.values[0])
    else:
        fill_color = 'gray'  # 데이터 없을 때
    return {
        'fillColor': fill_color,
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

# GeoJson으로 색상 직접 입히기
folium.GeoJson(
    seoul_geo,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['gu'], aliases=['구'])
).add_to(m)

# 색상 범례 추가
color_scale.caption = '서울시 주점 수'
color_scale.add_to(m)

# 지도 저장
m.save("seoul_colors_pubs.html")


In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import glob

file_people = r'E:\safety_sim\dataset\인구데이터\자치구별+연령별+인구(추계인구)_20250415145820.xlsx'

df_people = pd.read_excel(file_people)

e:\Anaconda\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [2]:
columns_2023 = [col for col in df_people.columns if '2023' in str(col)]
columns_2024 = [col for col in df_people.columns if '2024' in str(col)]
columns_2025 = [col for col in df_people.columns if '2025' in str(col)]

# Create separate DataFrames for each year
df_2023 = df_people[['자치구별(1)', '자치구별(2)', '성별(1)'] + columns_2023]
df_2024 = df_people[['자치구별(1)', '자치구별(2)', '성별(1)'] + columns_2024]
df_2025 = df_people[['자치구별(1)', '자치구별(2)', '성별(1)'] + columns_2025]

In [3]:
df_2023.fillna(method='ffill', inplace=True)
df_2024.fillna(method='ffill', inplace=True)
df_2025.fillna(method='ffill', inplace=True)

C:\Users\hyunj\AppData\Local\Temp\ipykernel_2628\390461079.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_2023.fillna(method='ffill', inplace=True)
C:\Users\hyunj\AppData\Local\Temp\ipykernel_2628\390461079.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023.fillna(method='ffill', inplace=True)
C:\Users\hyunj\AppData\Local\Temp\ipykernel_2628\390461079.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_2024.fillna(method='ffill', inplace=True)
C:\Users\hyunj\AppData\Local\Temp\ipykernel_2628\390461079.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

In [4]:

filtered2023_df = df_2023[df_2023["성별(1)"] == "계"].copy()
filtered2024_df = df_2024[df_2024["성별(1)"] == "계"].copy()
filtered2025_df = df_2025[df_2025["성별(1)"] == "계"].copy()

# 2번째 행을 맨 앞에 추가 (기존 인덱스 유지)
second_row = df_2023.iloc[1]  # 2번째 행 가져오기
filtered_df1 = pd.concat([pd.DataFrame([second_row]), filtered2023_df.drop(filtered2023_df.index[1])], ignore_index=True)

filtered_df1.head(2)


,자치구별(1),자치구별(2),성별(1),2023,2023.1,2023.2,2023.3,2023.4,2023.5,2023.6,...,2023.11,2023.12,2023.13,2023.14,2023.15,2023.16,2023.17,2023.18,2023.19,2023.20
0,자치구별(1),자치구별(2),성별(1),소계,0~4세,5~9세,10~14세,15~19세,20~24세,25~29세,...,50~54세,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95세 이상+
1,합계,소계,계,9400365,218633,298048,340540,374053,599242,825754,...,794228,692659,727214,598063,413958,309321,221134,102723,33576,8541


In [5]:
filtered2023_df.head(3)

,자치구별(1),자치구별(2),성별(1),2023,2023.1,2023.2,2023.3,2023.4,2023.5,2023.6,...,2023.11,2023.12,2023.13,2023.14,2023.15,2023.16,2023.17,2023.18,2023.19,2023.20
2,합계,소계,계,9400365,218633,298048,340540,374053,599242,825754,...,794228,692659,727214,598063,413958,309321,221134,102723,33576,8541
5,합계,종로구,계,146684,2405,3442,4290,6658,13308,14680,...,12344,11233,11419,8851,5999,5003,4195,2070,696,199
8,합계,중구,계,126473,2856,3067,2741,3805,8551,12343,...,10034,9395,9988,8371,5628,4516,3420,1659,581,156


In [6]:
# 그룹별 합산 (2023.1~2023.2 / 2023.3~2023.4 ... 2023.11~2023.12)
filtered2023_df["2023아동"] = filtered2023_df[["2023.1", "2023.2"]].sum(axis=1)
filtered2023_df["2023청소년"] = filtered2023_df[["2023.3", "2023.4"]].sum(axis=1)
filtered2023_df["2023성인"] = filtered2023_df[["2023.5", "2023.6", "2023.7", "2023.8", "2023.9", "2023.10", "2023.11", "2023.12"]].sum(axis=1)

filtered2024_df["2024아동"] = filtered2024_df[["2024.1", "2024.2"]].sum(axis=1)
filtered2024_df["2024청소년"] = filtered2024_df[["2024.3", "2024.4"]].sum(axis=1)
filtered2024_df["2024성인"] = filtered2024_df[["2024.5", "2024.6", "2024.7", "2024.8", "2024.9", "2024.10", "2024.11", "2024.12"]].sum(axis=1)


filtered2025_df["2025아동"] = filtered2025_df[["2025.1", "2025.2"]].sum(axis=1)
filtered2025_df["2025청소년"] = filtered2025_df[["2025.3", "2025.4"]].sum(axis=1)
filtered2025_df["2025성인"] = filtered2025_df[["2025.5", "2025.6", "2025.7", "2025.8", "2025.9", "2025.10", "2025.11", "2025.12"]].sum(axis=1)


# 필요 없는 열 제거 (기존 2023.x 열 삭제)
filtered2023_df = filtered2023_df[["자치구별(2)", "2023아동", "2023청소년", "2023성인"]]
filtered2023_df = filtered2023_df[filtered2023_df["자치구별(2)"] != "소계"]

filtered2024_df = filtered2024_df[["자치구별(2)", "2024아동", "2024청소년", "2024성인"]]
filtered2024_df = filtered2024_df[filtered2024_df["자치구별(2)"] != "소계"]

filtered2025_df = filtered2025_df[["자치구별(2)", "2025아동", "2025청소년", "2025성인"]]
filtered2025_df = filtered2025_df[filtered2025_df["자치구별(2)"] != "소계"]
filtered2024_df.head(2)


,자치구별(2),2024아동,2024청소년,2024성인
5,종로구,5495,10025,91375
8,중구,5480,6218,80111


In [7]:
merged_df = pd.merge(filtered2023_df, filtered2024_df, on="자치구별(2)", how="left")
merged_df = pd.merge(merged_df, filtered2025_df, on="자치구별(2)", how="left")
merged_df.head(3)

,자치구별(2),2023아동,2023청소년,2023성인,2024아동,2024청소년,2024성인,2025아동,2025청소년,2025성인
0,종로구,5847,10948,91457,5495,10025,91375,5054,9188,89829
1,중구,5923,6546,79685,5480,6218,80111,5110,6017,79581
2,용산구,12350,13313,138359,11027,12795,131308,9913,12421,128364


In [8]:
# 아동, 청소년, 성인별로 합산
df_sum = pd.DataFrame({
    '자치구별': merged_df['자치구별(2)'],
    '아동 합계': merged_df[['2023아동', '2024아동', '2025아동']].sum(axis=1),
    '청소년 합계': merged_df[['2023청소년', '2024청소년', '2025청소년']].sum(axis=1),
    '성인 합계': merged_df[['2023성인', '2024성인', '2025성인']].sum(axis=1)
})

df_sum.iloc[:, 1:] = df_sum.iloc[:, 1:].apply(lambda x: x / 3)
df_sum

,자치구별,아동 합계,청소년 합계,성인 합계
0,종로구,5465.333333,10053.666667,90887.0
1,중구,5504.333333,6260.333333,79792.333333
2,용산구,11096.666667,12843.0,132677.0
3,성동구,15960.333333,17021.333333,174413.0
4,광진구,14359.333333,22921.666667,222855.666667
5,동대문구,17362.666667,22580.0,223088.333333
6,중랑구,18024.0,23061.333333,221077.666667
7,성북구,22631.666667,34283.666667,263229.666667
8,강북구,10694.333333,18132.0,158048.0
9,도봉구,13042.666667,21158.666667,165339.666667


In [9]:
import json
import folium
import pandas as pd
from branca.colormap import LinearColormap

# 색상 단계 정의
colors = ['red', 'lightcoral', 'orange', 'blue', 'lightblue']
color_scale = LinearColormap(colors=colors,
                             vmin=df_sum['청소년 합계'].min(),
                             vmax=df_sum['청소년 합계'].max())

# GeoJSON 파일 로드
geojson_path = r'E:\safety_sim\dataset\서울특별시.json'
with open(geojson_path, encoding='utf-8') as f:
    seoul_geo = json.load(f)

# GeoJSON에 'gu' 필드 추가
for feature in seoul_geo['features']:
    adm_nm = feature['properties']['adm_nm']  # 예: '서울특별시 성동구 행당동'
    gu_name = adm_nm.split()[1]  # '성동구'
    feature['properties']['gu'] = gu_name

# CCTV 데이터에서 자치구 필터링
geo_gu_list = {f['properties']['gu'] for f in seoul_geo['features']}
gu_counts_filtered = df_sum[df_sum['자치구별'].isin(geo_gu_list)]

# folium 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

# 스타일 함수 정의 (색상 적용)
def style_function(feature):
    gu_name = feature['properties']['gu']
    value = gu_counts_filtered.loc[gu_counts_filtered['자치구별'] == gu_name, '청소년 합계']
    if not value.empty:
        fill_color = color_scale(value.values[0])
    else:
        fill_color = 'gray'  # 데이터 없을 때
    return {
        'fillColor': fill_color,
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

# GeoJson으로 색상 직접 입히기
folium.GeoJson(
    seoul_geo,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['gu'], aliases=['자치구별'])
).add_to(m)

# 색상 범례 추가
color_scale.caption = '서울시 청소년 수'
color_scale.add_to(m)

# 지도 저장
m.save("seoul_colors_Teen.html")


In [10]:
import json
import folium
import pandas as pd
from branca.colormap import LinearColormap

# 색상 단계 정의
colors = ['red', 'lightcoral', 'orange', 'blue', 'lightblue']
color_scale = LinearColormap(colors=colors,
                             vmin=df_sum['아동 합계'].min(),
                             vmax=df_sum['아동 합계'].max())

# GeoJSON 파일 로드
geojson_path = r'E:\safety_sim\dataset\서울특별시.json'
with open(geojson_path, encoding='utf-8') as f:
    seoul_geo = json.load(f)

# GeoJSON에 'gu' 필드 추가
for feature in seoul_geo['features']:
    adm_nm = feature['properties']['adm_nm']  # 예: '서울특별시 성동구 행당동'
    gu_name = adm_nm.split()[1]  # '성동구'
    feature['properties']['gu'] = gu_name

# CCTV 데이터에서 자치구 필터링
geo_gu_list = {f['properties']['gu'] for f in seoul_geo['features']}
gu_counts_filtered = df_sum[df_sum['자치구별'].isin(geo_gu_list)]

# folium 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

# 스타일 함수 정의 (색상 적용)
def style_function(feature):
    gu_name = feature['properties']['gu']
    value = gu_counts_filtered.loc[gu_counts_filtered['자치구별'] == gu_name, '아동 합계']
    if not value.empty:
        fill_color = color_scale(value.values[0])
    else:
        fill_color = 'gray'  # 데이터 없을 때
    return {
        'fillColor': fill_color,
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

# GeoJson으로 색상 직접 입히기
folium.GeoJson(
    seoul_geo,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['gu'], aliases=['자치구별'])
).add_to(m)

# 색상 범례 추가
color_scale.caption = '서울시 아동 수'
color_scale.add_to(m)

# 지도 저장
m.save("seoul_colors_baby.html")


In [11]:

filtered2023_df = df_2023[df_2023["성별(1)"] == "여자"].copy()
filtered2024_df = df_2024[df_2024["성별(1)"] == "여자"].copy()
filtered2025_df = df_2025[df_2025["성별(1)"] == "여자"].copy()

df_filtered1 = filtered2025_df.iloc[:, [1, 3]]
df_filtered2 = filtered2024_df.iloc[:, [1, 3]]
df_filtered3 = filtered2023_df.iloc[:, [1, 3]]


merged_df = pd.merge(df_filtered1, df_filtered2, on="자치구별(2)", how="left")
merged_df = pd.merge(merged_df, df_filtered3, on="자치구별(2)", how="left")
merged_df.head(3)




,자치구별(2),2025,2024,2023
0,소계,4852072,4873178,4865189
1,종로구,75747,76714,76680
2,중구,66730,66639,66051


In [12]:
# 아동, 청소년, 성인별로 합산
df_sum = pd.DataFrame({
    '자치구별': merged_df['자치구별(2)'],
    '여성 합계': merged_df[['2023', '2024', '2025']].sum(axis=1),
})
df_sum



df_filtered = df_sum[df_sum["자치구별"] != "소계"]
df_filtered.iloc[:, 1:] = df_filtered.iloc[:, 1:].apply(lambda x: x / 3)

df_filtered

C:\Users\hyunj\AppData\Local\Temp\ipykernel_2628\3947273389.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.iloc[:, 1:] = df_filtered.iloc[:, 1:].apply(lambda x: x / 3)


,자치구별,여성 합계
1,종로구,76380.333333
2,중구,66473.333333
3,용산구,109025.333333
4,성동구,143835.666667
5,광진구,179964.0
6,동대문구,184076.0
7,중랑구,193043.0
8,성북구,226642.666667
9,강북구,145988.333333
10,도봉구,155051.666667


In [14]:
import json
import folium
import pandas as pd
from branca.colormap import LinearColormap

# 색상 단계 정의
colors = ['red', 'lightcoral', 'orange', 'blue', 'lightblue']
# 색상 범위 수동 조정
color_scale = LinearColormap(colors=colors,
                             vmin=df_sum['여성 합계'].quantile(0.1),  # 10th percentile로 범위 설정
                             vmax=df_sum['여성 합계'].quantile(0.9))  # 90th percentile로 범위 설정


# GeoJSON 파일 로드
geojson_path = r'E:\safety_sim\dataset\서울특별시.json'
with open(geojson_path, encoding='utf-8') as f:
    seoul_geo = json.load(f)

# GeoJSON에 'gu' 필드 추가
for feature in seoul_geo['features']:
    adm_nm = feature['properties']['adm_nm']  # 예: '서울특별시 성동구 행당동'
    gu_name = adm_nm.split()[1]  # '성동구'
    feature['properties']['gu'] = gu_name

# CCTV 데이터에서 자치구 필터링
geo_gu_list = {f['properties']['gu'] for f in seoul_geo['features']}
gu_counts_filtered = df_sum[df_sum['자치구별'].isin(geo_gu_list)]

# folium 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

# 스타일 함수 정의 (색상 적용)
def style_function(feature):
    gu_name = feature['properties']['gu']
    value = gu_counts_filtered.loc[gu_counts_filtered['자치구별'] == gu_name, '여성 합계']
    if not value.empty:
        fill_color = color_scale(value.values[0])
    else:
        fill_color = 'gray'  # 데이터 없을 때
    return {
        'fillColor': fill_color,
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

# GeoJson으로 색상 직접 입히기
folium.GeoJson(
    seoul_geo,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['gu'], aliases=['자치구별'])
).add_to(m)

# 색상 범례 추가
color_scale.caption = '서울시 여성 수'
color_scale.add_to(m)

# 지도 저장
m.save("seoul_colors_women.html")
